In [ ]:
from scipy.signal import savgol_filter
from gym import spaces
from collections import deque
import json
import gym
import shlex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import scipy.stats as stats
import re
import time


In [ ]:
def loadSagaJson(path):
    with open(path, "r", encoding="utf-8") as file:
        l = []
        for i in file:
           l.append(json.loads(i)) 
    return l
def sigmoid(x):
    #這邊做了一個避免數字跑出特定範圍的東西
    #這邊承襲原本的版本 如果順利之後可以考慮用原版的去跑跑看
    x =np.clip(x,-500,5.0)
    return 1/(1+np.exp(-x))

def softmax(x, temperature=1.0):
    #同上 先用用看別人的順利再跑跑看不做限制的
    x = np.clip(x, -500, 500)
    x_max = np.max(x)
    exp_x = np.exp((x - x_max) / temperature)
    sum_exp_x = np.sum(exp_x)
    if sum_exp_x == 0:
        return np.ones_like(x) / len(x)
    return exp_x / sum_exp_x

## RL
這邊設定的類別有兩種，一是「環境」、二是「模型」(或是RL的演算法)
環境取決的載入哪一種紀錄檔案、資料集。如網路封包日誌就會看日誌用甚麼協定、一次傳輸多少bits等訊息，而Audit Log(審計日誌)則會看說使用哪個服務和做了什麼動作。
RL模型則是看要套用哪種演算法和技術
#### 環境設置

In [ ]:

class AuditLogEnv(gym.Env):
    def __init__(self,logFile="G16.json"):
        # 初始化環境
        ## 基礎設定
        self.logFile=logFile
        #logguardQ有設置size但很明顯我要處理的檔案大很多因此不考慮
        self.stateDim=100 #代表傳入的狀態Dimension有100維
        self.actionDim=4 #代表有四種動作
        ## 接著是載入某份log(這邊預設維G16.json，因為它檔案很小)
        self.logs=self.loadAuditLogs(logFile)#這邊載入會使用 
        if len(self.logs) == 0:#沒有成功載入任何 AuditLog的話會出現錯誤
            raise ValueError(f"No valid log entries found in {logFile}.")
        self.attackPattern=[]#這邊logguardQ給了一個輸入維度在哪種情況下可能是攻擊，但我這邊還不確定怎麼做
        self.noiseLvl=0.05 # 一個加入可能的躁點大概佔幾趴 不確定會不會用到
        ## 以下是代理每次動作後看做了哪些事可能會更新某些值
        self.ipCounts={} # 由於AuditLog也有ip相關可能會用到
        self.logIndex=0 # 看到第幾條
        self.stepCount=0 # 現在走了幾步 
        self.anomalyCount=0
        self.visitedStates=set() # 走訪過那些狀態
        self.processedLogs=set()# 已經處理了哪些東西 作用域: 單次回合 (Single Episode)
        self.processedLogsGlobal=set() #作用域: 整個環境的生命週期 (Entire Lifetime)。
        self.featureRanges={}# 把記錄到的某些數值縮放到某個數字區間 為了提高訓練效率和穩定性。 不確定會不會用到
        self.featureWeights={}# input的每個東西的權重，不確定要怎麼分配但先放著
        
        ## 常見弱點出現在哪些東西上
        vulnName=[]#常見的有問題的AuditLog可能的特徵 但這邊什麼都沒放 才剛架起來
        attackParamsValues=[]#logGuardQ使用常見的 XSS攻擊會出現的字串特徵 跟上面很像
        attackRegEx=[]# 仿造上面 這邊則用RegEx做做看 一樣還沒放入任何東西
        # 接下來是先載入一次這次回合的(Episode)的AuditLog 並且檢查有多少是非benign
        ## 它的主要功能是在環境載入完所有日誌後，對這些日誌進行初步的分析和統計，以瞭解數據集中異常日誌的「真實分佈」
        ###這邊我還沒改好
        for _, logEntry in self.logs.iterrows():
            if self._is_anomaly(logEntry):
                self.anomalyCount += 1
        print(f"Loaded {len(self.logs)} valid log entries from {logFile}.")
        print(f"Anomaly distribution: {self.anomalyCount}/{len(self.logs)} ({self.anomalyCount/len(self.logs)*100:.1f}% anomalies)")
        self.reset()
    def loadLogs(self,logFile,dataset:str="SAGA"):
        #預設載入SAGA資料集
        #這邊是載入AuditLog的地方，要做成pandas.DataFrame
        listColumns=['relation','timestamp','label','srcNode.UUID','srcNode.Name','srcNode.Image','srcNode.Cmdline','srcNode.Type','srcNode.Pid','dstNode.UUID','dstNode.Key','dstNode.Type','dstNode.Value','dstNode.Name','dstNode.Srcaddress','dstNode.Dstaddress','dstNode.Port','dstNode.Image','dstNode.Cmdline','dstNode.Pid','dstNode','dstNode.Path']#有甚麼樣的欄位取決於現在讀哪一個資料集
        listLogs=[]
        #讀取檔案
        try:
            with open(logFile,"r",encoding='utf-8')as file:
                for line in file:
                    listLogs.append(json.loads(line))
                    """
                    這邊先沒有用到但先不刪除
                    if lens(part) >=1:
                        listLogs.append(listParts[:11])
                    else:
                        print(f"skipping malformed line:{line.strip()},except 11 fields,got{len(listParts)}")
                        """
            pd.json_normalize(listLogs)
            if not listLogs:
                print(f"Warning:no valid log entries found in {logFile}.")
        except FileNotFoundError:
            raise FileNotFoundError(f"File {logFile} not found.")
        except Exception as e:
            raise Exception({f"Error loading file {logFile}: {str(e)}"})
        # 轉換成DataFrame
        df=pd.DataFrame(listLogs,columns=listColumns)
        # 下面則是要把某些部分有固定種類的東西轉換成數值表示
        # 舉例df['status_code'] = pd.to_numeric(df['status_code'], errors='coerce')
        # 特徵擷取分為四種
        #   無大小關係的一組數字(舉例：IP)：計算出現頻率 或 映射到不同數字上
        #   有大小關係的數字(舉例：傳送檔案或字串的大小)：進行歸一化處理(縮放到特定區間)
        #   字串：比對是否與可疑字串有關
        df['relation'] = pd.to_numeric(df['relation'], errors='coerce')
        return df
    def isAnomaly(self,logEntry):
        #對答案
        #這邊logguardQ的實作方法是給了一些很刻意的線索然後比對線索
        #我的實作方式是直接看標籤是不是benign
        #當然我會希望可以做得更好但先這樣
        return lambda logEntry: logEntry.get('label') != "benign"
    def extractFeature(self,logEntry):
        #1 擷取logEntry的各項Value存到各個變數
        #2 特徵標準化以及加權
        #3 特徵向量組合(將所有特徵封裝組合成一個numpy陣列)
        normalizedFeatures=np.zeros(features)
        for i,key in enumerate([0,0,0,00]):
            #最後再將所有特徵再次縮放到0,1之間
            minVal,maxVal=self.featureRanges(key)
            normalizedFeatures[i]=np.clip((features[i]-minVal)/(maxVal-minVal+1e-10),0,1)*self.featureWeights[key]
        return normalizedFeatures
    def reset(self):
        #1 重製游標/指標
        self.stepCount-0
        #2 清空歷史決策紀錄
        self.processedLogs=set()
        #3 初始化統計用變數
        #4 隨機選一段要跑的Audit log
        #5 回傳初始的State
        self.stepCount=0
        self.processedLogs=set()
        if len(self.logs) > 0:
            unprocessedIndices=[i for i in range(len(self.logs))if i not in self.processedLogsGlobal]
            if unprocessedIndices:
                self.logIndex=random.choice(unprocessedIndices)
            else:
                self.logIndex=np.random.randint(0,len(self.logs))
        else:
            raise IndexError(f"Log Index{self.logIndex} out of range [0,{len(self.logs)}]")
        self.ipCounts={}
        self.visitedStates=set()
        self.state=self.getState()
        return self.state
    def getState(self):
        if 0 <= self.logIndex < len(self.logs):
            logEntry=self.logs.iloc[self.logIndex]
            # ip
            ip=logEntry['ip']
            self.ipCounts[ip]=self.ipCounts.gets(ip,0)+1
            if len(self.ipCounts)>100
            ipOldest=list(self.ipCounts.keys())[0]
            self.ipCounts[ipOldest]-=1
            if self.ipCounts[ipOldest]==0:
                del self.ipCounts[ipOldest]
            state=self.extractFeature(logEntry,self.ipCounts)
            self.visitedStates.add(tuple(state))
        else:
            raise IndexError(f"Log Index {self.logIndex} out of range [0,{len(self.logs)}]")
        return
    def  updateNoise(self):
        self.noiseLvl=0.05
    def step(self,action):
        # 一步
        self.stepCount+=1
        if 0<=self.logIndex <len(self.logs):
            logEntry=self.logs.iloc[self.logIndex]
            isAnomaly=self.isAnomaly(logEntry)
            if action == 0 and isAnomaly:
                score = 25.0
                done = False
            elif action == 0 and not isAnomaly:
                score = -15.0
                done = False
            elif action != 0 and isAnomaly:
                score = 2.0
                done = False
            if np.random.rand() <self.noiseLvl:
                score -= 0.5
            self.processedLogs.add(self.logIndex)
            self.logIndex = (self.logIndex + 1) % len(self.logs)
            self.state=self.getState()
            done = self.stepCount>=5 or len(self.processedLogsGlobal)>= self.anomalyCount4
            return self.state,score,done,isAnomaly
        else:
            raise IndexError(f"Log Index {self.logIndex} out of range [0,{len(self.logs)}]")

        
    

#### 決策機設定

In [ ]:
"""
DQN的運作流程
迴圈
    1.環境重製
    2.根據現在狀態選擇一個動作
    3.透過動作與環境運作之後
    4.根據這次互動之後更新權重

"""
class DQN:
    """
    
    """
    def __init__(self,stateDim,actionDim,hiddenDim=16):
        ##同步狀態維度(狀態特徵數量)和動作維度
        self.stateDim=stateDim
        self.actionDim=actionDim
        ##  設定深度學習的網路層數以及參數數量
        self.hiddenDim=16
        self.weightS1=np.random.randn(stateDim,hiddenDim)*0.01
        self.weightS2=np.random.randn(hiddenDim,actionDim)*0.01
        self.memory= deque(maxlen=1000)
        ##  各項演算法中的代數設定
        self.gamma =0.99 #折扣因子
        self.epsilon=1.0#  ε-greedy的 ε值 判斷模型該「探索」還是「利用知識」
        self.epsilonMin=0.01# ε的最小值
        self.epsilonDecay=0.998#  ε-greedy的衰減值 模型會逐漸從「探索」轉向利用知識
        self.learnRate=0.002# 學習率
        self.varianceHistory=[]# 紀錄每個回合(episode) 代理所獲得分數的標準差
        self.scoreStates=[]#用於儲存分數的平均值和標準差
        self.episode=0
        self.actionHistory=[]
        
    def chooseAction(self,state,visitedStates):
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.actionDim)
        else:
            qValues=self.forward(state)
            action =np.argmax(qValues)
        curiosityBonus=1.0 if tuple(state) not in visitedStates else 0.0
        ## 如果採取一個新動作的時候會觸發一個好奇心獎勵值
        self.actionHistory.append(action)
        return action,curiosityBonus
    def forward(self,state):
        #用矩陣乘法把狀態乘上第一層權重再用sigmoid(隱藏層的活化函數)
        hidden = sigmoid(np.dot(state,self.weightS1))
        #再來矩陣乘法乘上第二層權重
        #向量中的每一個值就是神經網路預測的在當狀態下採取特定動作的Q值
        return np.dot(hidden,self.weightS2)
    def updateWeights(self,state,action,reward,prevState,curiosityBonus):
        nextQValue=self.forward(state)
        target=reward+self.gamma*np.max(nextQValue)+curiosityBonus
        currentQ=self.forward(prevState)[action]
        delta=target-currentQ
        hidden=sigmoid(np.dot(prevState,self.weightS1))
        deltaOutput=np.zeros(self.actionDim)
        deltaOutput[action]=delta
        grade2=np.outer(prevState,deltaOutput)
        deltaHidden=np.dot(self.weightS2,deltaOutput)*hidden*(1-hidden)
        grade1=np.outer(prevState,deltaHidden)
        self.weightS1+=self.learnRate*grade1
        self.weightS2+=self.learnRate*grade2
        self.memory.append((prevState,action,reward,state))
        self.epsilon=max(self.epsilonMin,self.epsilon*self.epsilonDecay)
        self.scoreStates['std']=np.sqrt(self.scoreStates.get('std',0)**2*0.99+(reward-self.scoreStates['mean'])**2*0.01)
        self.varianceHistory.append(self.scoreStates['std'])
    def reportDetection():
        #這個不知道在幹嘛的
        pass

"""

"""
class PPO:
    def __init__(self):
        return
    def chooseAction():
        return
    def forward():
        return
    def updateWeights():
        return
    def reportDetection():
        pass

#### 功能

In [ ]:
def evaluateStatisticalSignificance():
    #針對不同的RL模型進行更嚴謹的性能和統計學比較
    #實作與功能
    # 先為每個模型的得分計算平均數 中位數 標準差 常態分佈檢定
    # 再來選擇並且統計檢定 比較每兩個東西模型的表現
    # 最後印出剩下的結果
    return
def runSimulation(model,env,modelName,episodes,maxStep):
    #1 初始化變數
    scores=[]
    f1Scores=[]
    steptoDetection=[]
    detections=0
    truePositives=0
    falsePositives=0
    trueNegatives=0
    falseNegatives=0
    totalAnomalies=0
    totalStart=time.time()
    cumulativeTp=[]
    #2 迴圈
    for episode in range(episodes):
        #2.1 提前停止條件
        if len(env.processedLogsGlobal)>= env.anomalyCount:
            #如果已經跑完全部的異常的話即停止
            print(f"{modelName} stopped early at episode {episode}: All {env.anomalyCount} anomalies processed.")
            break
        #每個episode開始 重置環境 清空episode統計
        state = env.reset()
        totalScore = 0
        steps=  0
        done = False
        episodeTruePositive=0
        episodeAnomalies=0
        actionCounts=np.zeros(4)
        #每一百步會更新一次躁點避免死記硬背
        if episode % 100 ==0:
            env.updateNoise()
        
    return
def displayResult(model="",dictResult:dict={}):
    precision=dictResult['tp']/(dictResult['tp']+dictResult['fp'])
    recall=dictResult['tp']/(dictResult['tp']+dictResult['fn'])
    f1Score=2*(precision*recall)/(precision+recall)
    successfulSteps=[s for s in dictResult['steps'][-100:]if s<5]
    subTitle=f"""
{model} Final Result
=====================
precision:{precision}
recall:{recall}
detection:{dictResult['detection']/len(dictResult['scores'])}({dictResult['detection']}/
{len(dictResult['scores'])*100:.1f}%)
true positive:{dictResult['tp']}
false positive:{dictResult['fp']}
true negative:{dictResult['tn']}
false negative:{dictResult['fn']}
total anomalies encountered:{dictResult['anomalies']}
({dictResult['anomalies']/dictResult['tp']+dictResult['tn']+dictResult['fp']+dictResult['fn']*100:.1f}% of steps)
Mean Reward (all episodes): {np.mean(dictResult['scores']):.4f} ± {np.std(dictResult['scores']):.4f}
Mean F1-Score (all episodes): {np.mean(f1Score):.4f} ± {np.std(f1Score):.4f}
Mean Steps per episode (all episodes): {np.mean(dictResult['steps']):.1f} ± {np.std(dictResult['steps']):.1f}
Metrics for last 100 Episodes:
Mean Reward (last 100): {np.mean(dictResult['scores'][-100:]):.4f} ± {np.std(dictResult['scores'][-100:]):.4f}
Mean F1-Scores (last 100): {np.mean(f1Score[-100:]):.4f} ± {np.std(f1Score[-100:]):.4f}
Steps to Detection (last 100, successful episodes): {np.mean(successfulSteps):.1f} ± {np.std(successfulSteps):.1f}
"""if successfulSteps else "No Successful episodes"
    print(subTitle)
    return


# 主程式

In [ ]:
if __name__ == "__main__" :
    #先印出某個Audit Log檔案的前十行
    print("")
    np.random.seed(42)
    #環境設定初始化
    
    #RL模型初始化
    
    #回合(Episode)設定初始化
    
    #開始進行模擬
    
    #評估結果
    
    #印出結果